# Исследование надёжности заёмщиков

## Оглавление
<a id = "index"></a>

1. [Задание](#task)
1. [Настройка окружения](#env)
1. [Загрузка и первичное изучение данных](#first_look)
1. [Подготовка данных](#etl)
1. [Исследовательский анализ данных](#eda)
1. [Главные гипотезы](#main_hypotheses)
1. [Выводы и рекомендации](#conclusion)

## Задание
<a id = "task"></a>
[В оглавление](#index)

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Настройка окружения
<a id = "env"></a>
[В оглавление](#index)

In [1]:
pip install pymystem3

You should consider upgrading via the '/Users/maksim/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from collections import Counter

from pymystem3 import Mystem
m = Mystem()

# отображение одного знака после запятой
pd.options.display.float_format = '{:,.1f}'.format

## Загрузка и первичное изучение данных
<a id = "first_look"></a>
[В оглавление](#index)

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод по разделу

1. Требуется заполнение пропусков. Пропуски по столбцам **days_employed**,  **total_income** похожи на неслучайные, т.к. пропущенно одинаковое кол-во строк.
2. Типы данных подходят для анализа и перевод в иные типы не требуется. Обнаружены некорректные данные как минимум по столбцу 'days_employed'. Требуется детальный анализ состава данных по каждому столбцу.
3. Детальный анализ и выводы по каждому столбцу приведены в соответствующих шагах ниже. 

## Подготовка данных
<a id = "etl"></a>
[В оглавление](#index)

- Необходимо вести возрастные категории до этапа обработки пропусков. С учётом возрастных категорий обработка пропусков будет выполнена точнее. 
- Для ввода возрастных категорий необходимо провести анализ столбца ВОЗРАСТА и при необходимости исправить данные. 

#### Столбец 'income_type'

In [6]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

__Выводы:__ 
* "компаньон" — это семейное положение, поэтому нужно изменить на "безработный", 
* "в декрете" — это временный статус работающего, поэтому нужно изменить на "сотрудник".

In [7]:
# изменение "компаньон" на "безработный"
df.loc[df['income_type'] == 'компаньон', 'income_type'] = 'безработный'
#df['income_type'] = df['income_type'].replace('компаньон', 'безработный')

In [8]:
# изменение "в декрете" на "сотрудник"
df.loc[df['income_type'] == 'в декрете', 'income_type'] = 'сотрудник'
#df['income_type'] = df['income_type'].replace('в декрете', 'сотрудник')

In [12]:
# проверка после изменений
df['income_type'].value_counts()

сотрудник          11120
безработный         5087
пенсионер           3856
госслужащий         1459
предприниматель        2
студент                1
Name: income_type, dtype: int64

#### Столбец 'dob_years'

In [13]:
# проверка нулевого возраста или несовершеннолетия заёмщиков
df['dob_years'].describe()

count   21,525.0
mean        43.3
std         12.6
min          0.0
25%         33.0
50%         42.0
75%         53.0
max         75.0
Name: dob_years, dtype: float64

__Выводы:__
* Аномально высокого возраста заёмщиков нет. 
* Нулевой возраст недопустим. Необходимо заменить нулевой возраст средним в зависимости от ТИПА ЗАНЯТОСТИ.

In [14]:
# создание временного вспомогательного словаря заёмщиков с нулевым возрастом по ТИПУ ЗАНЯТОСТИ
income_type_dict = df[df['dob_years'] == 0]['income_type']
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop = True)
income_type_dict

0      пенсионер
1      сотрудник
2    безработный
3    госслужащий
Name: income_type, dtype: object

In [22]:
# замена нулевого возраста средним в зависимости от ТИПА ЗАНЯТОСТИ
for income_type in income_type_dict:
    median_age = df[(df['income_type'] == income_type) & 
                    (df['dob_years'] != 0)]['dob_years'].median()
    df.loc[(df['income_type'] == income_type) & 
           (df['dob_years'] == 0), 'dob_years'] = median_age
    
df['dob_years'] = df['dob_years'].astype("int")

In [23]:
# проверка наличия нулевого возраста после изменений
df['dob_years'].value_counts().sort_index().head()

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

In [24]:
# категоризация возраста (на основе Крылов А.А. Глава 15. Возрастные периоды развития человека)
def age_group(age):
    if age < 18:
        return 'несовершеннолетний'
    if age >= 18 and  age <= 25:
        return 'молодой'
    elif age > 25 and age <= 40:
        return 'средний'
    elif age > 40 and age <= 65:
        return 'выше среднего'
    else:
        return 'пожилой'

df['age_group'] = df['dob_years'].apply(age_group)

In [25]:
# создание вспомогательного словаря ВОЗРАСТНЫХ ГРУПП
age_group_dict = df['age_group']
age_group_dict = age_group_dict.drop_duplicates().reset_index(drop = True)
age_group_dict

0    выше среднего
1          средний
2          молодой
3          пожилой
Name: age_group, dtype: object

### Обработка пропусков

In [26]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
age_group              0
dtype: int64

In [27]:
# проверка пропущенных значений СТАЖА по типу занятости
df['days_employed'].isnull().groupby(df['income_type']).sum().astype('int')

income_type
безработный         508
госслужащий         147
пенсионер           413
предприниматель       1
сотрудник          1105
студент               0
Name: days_employed, dtype: int64

In [28]:
# проверка пропущенных значений ДОХОДА по типу занятости
df['total_income'].isnull().groupby(df['income_type']).sum().astype('int')

income_type
безработный         508
госслужащий         147
пенсионер           413
предприниматель       1
сотрудник          1105
студент               0
Name: total_income, dtype: int64

- Количество пропущенных записей по СТАЖУ и ДОХОДУ одинаково по одним и тем же группам занятости.
- Пропущенные значения СТАЖА и ДОХОДА по группам "госслужащий", "компаньон", "пенсионер", "сотрудник" необходимо заполнить медианными значениями по ТИПУ ЗАНЯТОСТИ (и ВОЗРАСТНОЙ ГРУППЕ).
- Необходимо отдельно проверить запись ПРЕДПРИНИМАТЕЛЬ.

__Обработка пропусков по записи ПРЕДПРИНИМАТЕЛЬ__ 

In [29]:
# запись ПРЕДПРИНИМАТЕЛЬ без дохода и стажа
df[(df['income_type'] == 'предприниматель') & 
   (df['days_employed'].isnull())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5936,0,nan,58,высшее,0,женат / замужем,0,M,предприниматель,0,nan,покупка жилой недвижимости,выше среднего


Запись ПРЕДПРИНИМАТЕЛЬ с пропущенными значениями СТАЖА и ДОХОДА содержит информацию о семейном положении, детях и возврате кредита. При этом медиану СТАЖА и ДОХОДА по группе ПРЕДПРИНИМАТЕЛЬ считать некорректно, т.к. с таким ТИПОМ ЗАНЯТОСТИ всего две записи. Принято решение об удалении записи.

In [30]:
# удаление записи ПРЕДПРИНИМАТЕЛЬ без дохода и стажа
df.drop(df[(df['income_type'] == 'предприниматель') & (df['days_employed'].isnull())].index , inplace = True)

In [31]:
# Также по данной записи пропущенные значения можно заполнить нулями.

# заполнение пропущенных значений по записи ПРЕДПРИНИМАТЕЛЬ без дохода и стажа
#df.loc[(df['income_type'] == 'предприниматель') & (df['days_employed'].isnull()), 'days_employed'] = 0
#df.loc[(df['income_type'] == 'предприниматель') & (df['total_income'].isnull()), 'total_income'] = 0

#### Обработка пропусков по столбцу 'days_employed'

In [32]:
# создание временного вспомогательного словаря
income_type_dict = df[df['days_employed'].isnull()]['income_type']
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop = True)

# замена пропущенного стажа — средним в зависимости от ТИПА ЗАНЯТОСТИ и ВОЗРАСТНОЙ ГРУППЫ
for income_type in income_type_dict:
    for age_group in age_group_dict:
        mean_days_employed = df[(df['income_type'] == income_type) & 
                                (df['age_group'] == age_group) & 
                                (df['days_employed'].isnull() == False)]['days_employed'].mean()
        df.loc[(df['income_type'] == income_type) & 
               (df['age_group'] == age_group) & 
               (df['days_employed'].isnull() == True) , 'days_employed'] = mean_days_employed

#### Обработка пропусков по столбцу "total_income" 

In [33]:
# проверка отрицательного дохода
df['total_income'].value_counts().sort_index().head()

20,667.3    1
21,205.3    1
21,367.6    1
21,695.1    1
21,895.6    1
Name: total_income, dtype: int64

In [34]:
# проверка аномально высокго дохода
df['total_income'].value_counts().sort_index().tail()

1,711,309.3    1
1,715,018.4    1
1,726,276.0    1
2,200,852.2    1
2,265,604.0    1
Name: total_income, dtype: int64

Артефактов в ДОХОДАХ не обнаружено.

In [35]:
# создание временного вспомогательного словаря ТИПОВ ЗАНЯТОСТИ с пропущенными значениями ДОХОДА
income_type_dict = df[df['total_income'].isnull()]['income_type']
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop = True)

# замена пропущенного дохода — медианным в зависимости от ТИПА ЗАНЯТОСТИ и ВОЗРАСТА
for income_type in income_type_dict:
    for age_group in age_group_dict:
        median_total_income = df[(df['income_type'] == income_type) & 
                                 (df['age_group'] == age_group) & 
                                 (df['total_income'].isnull() == False)]['total_income'].median()
        df.loc[(df['income_type'] == income_type) & 
               (df['age_group'] == age_group) & 
               (df['total_income'].isnull() == True) , 'total_income'] = median_total_income

#### Проверка обработки пропусков

In [36]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

Количество пропусков по столбцам **days_employed**,  **total_income** одинаковое.  Вероятно они возникли при объединении нескольких баз данных. Необходимо сообщить разработчикам.

### Замена типа данных

Типы данных подходят для анализа и перевод не требуется. Требуется изменение самих данных.

#### Столбец 'gender'

In [37]:
df['gender'].value_counts()

F      14236
M       7287
XNA        1
Name: gender, dtype: int64

In [38]:
df[(df['gender'] == 'XNA')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
10701,0,"-2,358.6",24,неоконченное высшее,2,гражданский брак,1,XNA,безработный,0,"203,905.2",покупка недвижимости,молодой


* В РФ пока нет возможности указать иной пол кроме М или Ж, поэтому неуказанный/неизвестный пол считается за опечатку. Единичную запись можно удалить, так как заёмщик с неизвестным полом только один.
* Пол указан на английском языке, в то время как остальные данные на русском. Принято решение о приведении к единому стилю выборки.

In [39]:
# удаление записей с неизвестным/неуказанным полом
df.drop(df[df['gender'] == 'XNA'].index , inplace = True)

# корректировка названий полов
df.loc[df['gender'] == "F", 'gender'] = 'Ж'
df.loc[df['gender'] == "M", 'gender'] = 'М'

In [40]:
# проверка
df['gender'].value_counts()

Ж    14236
М     7287
Name: gender, dtype: int64

#### Столбец 'days_employed'

In [41]:
# проверка среднего стажа работы по типу занятости
(df.groupby('income_type')['days_employed'].mean()).astype('int64')

income_type
безработный         -1951
госслужащий         -3395
пенсионер          364993
предприниматель      -520
сотрудник           -2328
студент              -578
Name: days_employed, dtype: int64

In [42]:
# проверка среднего стажа работы по типу занятости в годах
(df.groupby('income_type')['days_employed'].mean()/365).astype('int64')

income_type
безработный         -5
госслужащий         -9
пенсионер          999
предприниматель     -1
сотрудник           -6
студент             -1
Name: days_employed, dtype: int64

- Модули отрицательных значений стажа похожи на реальный стаж работы — __их нужно обратить__.
- У пенсионеров скорее всего в столбце СТАЖ возникло аномальное значение вместо реального стажа. Экспресс-гипотеза, что указаны года со сбитым форматом не подтвердилась, т.к. появились отрицательные значения года начала работа. __Принято решение: для пенсионеров указать  стаж с учётом начала работы в 20 лет и до пенсии: для женщин — 55 лет, для мужчин — 60 лет.__

#### Корректировка СТАЖА для всех, кроме пенсионеров

In [43]:
# обращение отрицательных значений
df.loc[df['income_type'] != 'пенсионер', 'days_employed'] = df.loc[df['income_type'] != 'пенсионер', 'days_employed'].abs()

#### Корректировка СТАЖА для пенсионеров

In [44]:
# замена СТАЖА по пенсионерам-женщинам
df.loc[(df['income_type'] == 'пенсионер') & (df['gender'] == 'Ж'), 'days_employed'] = 35*365

# замена СТАЖА по пенсионерам-мужчинам
df.loc[(df['income_type'] == 'пенсионер') & (df['gender'] == 'М'), 'days_employed'] = 40*365

### Перевод дней стажа в целочисленный вид 

In [45]:
df['days_employed'] = df['days_employed'].astype('int64')

- Типы данных подходят для анализа и перевод не требуется.
- Необходимо проработать с разработчиками причины некорректного заполенения СТАЖА:
    * отрицательного для всех, кроме пенсионеров;
    * аномального для пенсионеров.

In [46]:
df.dtypes

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_group            object
dtype: object

#### Столбец 'children'

In [47]:
# проверка дубликатов по столбцу ДЕТИ
df['children'].value_counts()

 0     14147
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Присутствуют артефакты — отрицательные значения и у 76 заёмщиков ровно по 20 детей :
* Отрицательные значения детей: вряд ли это умершие дети, поэтому считается за опечатку. Группу заёмщиколв стоит обеденить с группой, у кого 1 ребёнок.
* Выбивающееся значение "20 детей": скорее всего опечатка, так как после 5 детей идёт сразу 20 и нет промежуточных значений, а само значение "20 детей" встречается не один раз. Группу заёмщиков стоит объеденить к группой, у кого 2 ребёнка.

In [48]:
# корректировка отрицательных значений детей
df.loc[df['children'] == -1, 'children'] = 1

In [49]:
# корректировка значений "20 детей"
df.loc[df['children'] == 20, 'children'] = 2

In [50]:
# проверка
df['children'].value_counts()

0    14147
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Столбец 'education'

In [51]:
df['education'].value_counts()

среднее                13750
высшее                  4717
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [52]:
df['education_id'].value_counts()

1    15233
0     5259
2      743
3      282
4        6
Name: education_id, dtype: int64

Данные заполнены несистемно — требуется приведние в строчный вид.

In [53]:
# df.duplicated

In [54]:
# приведение к строчному виду
df['education'] = df['education'].str.lower()

In [55]:
# проверка
if df['education'].value_counts().sum() == df['education_id'].value_counts().sum():
    print('OK')

OK


#### Столбец 'family_status'

In [56]:
df['family_status'].value_counts()

женат / замужем          12379
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [57]:
df['family_status_id'].value_counts()

0    12379
1     4176
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

* Данные требуют приведния в строчный вид с привязкой к соответствующему идентификатору.
* Согласно Семейному кодексу РФ, незарегистрированное совместное проживание мужчины и женщины,т.е. __гражданский брак__, не порождает брачных прав и обязанностей, в части не касаемой детей.
* Перевод категории "гражданский брак" в "не женат / не замужем" следует обсудить с юр. отделом.

In [58]:
# приведение к строчному виду
df['family_status'] = df['family_status'].str.lower()

In [59]:
# проверка
if df['family_status'].value_counts().sum() == df['family_status_id'].value_counts().sum():
    print('OK')

OK


#### Столбец 'debt'

In [60]:
df['debt'].value_counts().sort_index().head()

0    19782
1     1741
Name: debt, dtype: int64

Данные в порядке, дополнительных действий не требуется.

### Обработка дубликатов

In [61]:
df.duplicated().sum()

71

In [62]:
df[df.duplicated() == True]

#df = df.drop_duplicates().reset_index()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
2849,0,2982,41,среднее,1,женат / замужем,0,Ж,сотрудник,0,"143,303.0",покупка жилья для семьи,выше среднего
3290,0,12775,58,среднее,1,гражданский брак,1,Ж,пенсионер,0,"121,697.6",сыграть свадьбу,выше среднего
4182,1,1878,34,высшее,0,гражданский брак,1,Ж,сотрудник,0,"144,087.7",свадьба,средний
4851,0,12775,60,среднее,1,гражданский брак,1,Ж,пенсионер,0,"121,697.6",свадьба,выше среднего
5557,0,12775,58,среднее,1,гражданский брак,1,Ж,пенсионер,0,"121,697.6",сыграть свадьбу,выше среднего
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,12775,64,среднее,1,женат / замужем,0,Ж,пенсионер,0,"121,697.6",дополнительное образование,выше среднего
21032,0,12775,60,среднее,1,женат / замужем,0,Ж,пенсионер,0,"121,697.6",заняться образованием,выше среднего
21132,0,2982,47,среднее,1,женат / замужем,0,Ж,сотрудник,0,"143,303.0",ремонт жилью,выше среднего
21281,1,1878,30,высшее,0,женат / замужем,0,Ж,сотрудник,0,"144,087.7",покупка коммерческой недвижимости,средний


In [63]:
df = df.drop_duplicates().reset_index()

### Вывод

Возникновение дубликатов связано с ошибками при ручном вводе информации и в отдельных случаях заполенением информацией, неподходящей по смыслу. Необходимо проработать данный вопрос с персоналом, ответственным за непосредственный ввод данных.

In [64]:
df.duplicated().sum()

0

### Лемматизация

In [65]:
# создание пустого словаря лемм
lemmas_dict_temp = []

In [66]:
# выборка уникальных целей получения кредита
purpose_dict = df['purpose'].unique()

In [67]:
# лемматизация целей получения кредита 
for purpose in purpose_dict:
    lemmas = m.lemmatize(purpose)
    for l in lemmas:
        if l !=' ' and l != '\n':
            lemmas_dict_temp.append(l)

print(Counter(lemmas_dict_temp))

# удаление дубликатов в словаре целей получения кредита
lemmas_dict_temp = list(dict.fromkeys(Counter(lemmas_dict_temp)))

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


In [68]:
# формирование основного словаря целей получения кредита для будущей категоризации
lemmas_dict = [
    ['недвижимость', 'операция с недвижимостью'],
    ['автомобиль', 'покупка автомобиля'], 
    ['образование', 'образование'],
    ['жилье', 'операция с недвижимостью'],
    ['свадьба', 'свадьба']]
lemmas_dict

[['недвижимость', 'операция с недвижимостью'],
 ['автомобиль', 'покупка автомобиля'],
 ['образование', 'образование'],
 ['жилье', 'операция с недвижимостью'],
 ['свадьба', 'свадьба']]

### Вывод

Основные цели получения кредита:
* операция с недвижимостью,
* покупка автомобиля,
* образование,
* свадьба.

### Категоризация данных

#### По уровню дохода 

__ОТРАБОТКА ЗАМЕЧАНИЯ:__ За основу категоризации взят подход из исследования Центр стратификационных исследований ИСП НИУ ВШЭ "Модели стратификации российского общества по доходам: подходы и результаты" по медианному разделению выборки:
* очень низкий <= 0,5 Median
* 0,5 Median < низкий <= 0,75 Median     
* 0,75 Median < средний <= 1,25 Median
* 1,25 Median < выше среднего <= 2 Median
* 2 Median < высокий <= 4 Median
* очень высокий <= 0,5 Median

In [69]:
median = df['total_income'].median()

In [70]:
def total_income_group(total_income):
    if total_income <= median*0.5:
        return 'очень низкий'
    elif total_income > median*0.5 and total_income <= median*0.75:
        return 'низкий'
    elif total_income > median*0.75 and total_income <= median*1.25:
        return 'средний'
    elif total_income > median*1.25 and total_income <= median*2:
        return 'выше среднего'
    elif total_income > median*2 and total_income <= median*4:
        return 'высокий'
    else:
        return 'очень высокий'

### Учтено.

In [71]:
# добавление нового столбца уровня дохода
df['total_income_group'] = df['total_income'].apply(total_income_group)

#### По цели получения кредита

In [72]:
# подпропрограмма 
def purpose_type_group(purpose):

# лемматизация цели получения кредита
    lemmas = m.lemmatize(purpose)

#  сравнение лемм со значениями из словаря лемм
    for l in lemmas:
        for l_dict in lemmas_dict:
            if l_dict[0] == l:
                return l_dict[1]

In [73]:
# категоризация по цели получения кредита 
df['purpose_type'] = df['purpose'].apply(purpose_type_group)

### Вывод



## Проверка гипотез
<a id = "main_hypotheses"></a>
[В оглавление](#index)

In [74]:
# основная информация о доходах заёмщиков в выборке
df.pivot_table(index=['income_type'], columns='total_income_group', values='total_income', aggfunc='count')

total_income_group,высокий,выше среднего,низкий,очень высокий,очень низкий,средний
income_type,,,,,,
безработный,639.0,"1,440.0",604.0,70.0,153.0,"2,173.0"
госслужащий,123.0,329.0,253.0,7.0,100.0,645.0
пенсионер,144.0,578.0,914.0,10.0,598.0,"1,585.0"
предприниматель,1.0,nan,nan,nan,nan,nan
сотрудник,661.0,"2,372.0","2,066.0",41.0,780.0,"5,165.0"
студент,nan,nan,1.0,nan,nan,nan


- У многих официально безработных доход средний и выше среднего. Скорее всего это самозянятые.
- У более чем половины пенсионеров также доход средний и выше среднего, скорее всего они продолжают работать и смогут отдавать кредит, но при выдаче нового нужно учитывать возраст. 

### Зависимость между наличием детей и возвратом кредита в срок

In [75]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от КОЛИЧЕСТВА ДЕТЕЙ
print(df.groupby(['children'])['debt'].mean().sort_values(ascending = False))

children
4   0.1
2   0.1
1   0.1
3   0.1
0   0.1
5   0.0
Name: debt, dtype: float64


- Заёмщики без детей в среднем наиболее дисциплинированы из всех, имеющих доход — только 7,5 % просрочили возврат кредита. 
- Заёмщики с 4 детьми наименее дисциплинированы — 9,8 % просрочили возврат кредита. Странно, всего один ребёнок, а такая разница с заёмщиками с 5 детьми.

In [76]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от КОЛИЧЕСТВА ДЕТЕЙ
df.pivot_table(index=['children'], columns='income_type', values='debt', aggfunc='mean')

income_type,безработный,госслужащий,пенсионер,предприниматель,сотрудник,студент
children,,,,,,
0,0.1,0.1,0.1,0.0,0.1,0.0
1,0.1,0.1,0.0,nan,0.1,nan
2,0.1,0.0,0.1,nan,0.1,nan
3,0.1,0.1,0.2,nan,0.1,nan
4,0.0,0.0,0.0,nan,0.1,nan
5,0.0,0.0,nan,nan,0.0,nan


- У безработных и госслужащих факт возврата кредита вовремя в целом несильно зависит от количества детей — около 7 %.
- У пенсионеров и сотрудников с увеличением количества детей растет доля невозврата кредита вовремя — до 17 и 13 % соответственно.
- Заёмщики с 5 детьми всегда возвращали долги вовремя. Видимо планы завести 5 детей подкреплены финансовой базой, которая в свою очередь достигается с хорошим навыком финансовой грамотности.

### Зависимость между семейным положением и возвратом кредита в срок

In [77]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от СЕМЕЙНОГО ПОЛОЖЕНИЯ
print(df.groupby(['family_status'])['debt'].mean().sort_values(ascending = False))

family_status
не женат / не замужем   0.1
гражданский брак        0.1
женат / замужем         0.1
в разводе               0.1
вдовец / вдова          0.1
Name: debt, dtype: float64


In [78]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от СЕМЕЙНОГО ПОЛОЖЕНИЯ
df.pivot_table(index=['children'], columns='family_status', values='debt', aggfunc='mean')

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
children,,,,,
0,0.1,0.1,0.1,0.1,0.1
1,0.1,0.1,0.1,0.1,0.1
2,0.1,0.1,0.1,0.1,0.1
3,0.1,0.0,0.1,0.1,0.1
4,0.0,0.0,0.0,0.1,0.5
5,nan,nan,0.0,0.0,nan


* Распределение зависимости невозврата кредита в срок от СЕМЕЙНОГО ПОЛОЖЕНИЯ довольно плавное — __с увеличением количества детей растёт риск невозврата кредита вовремя.__
* Вдовцы наиболее финансово дисциплинированны — в среднем просрочили возврат кредита 6,6 %. Не женатые/не замужние наименее дисциплинированны — 9,7 % невозвратов.
* __В зоне риска группы, живущие без партнёра.__

### Зависимость между уровнем дохода и возвратом кредита в срок

In [79]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от УРОВНЯ ДОХОДОВ
print(df.groupby(['total_income_group'])['debt'].mean().sort_values(ascending = False))

total_income_group
средний         0.1
низкий          0.1
выше среднего   0.1
высокий         0.1
очень низкий    0.1
очень высокий   0.1
Name: debt, dtype: float64


In [80]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от УРОВНЯ ДОХОДОВ
df.pivot_table(index=['children'], columns='total_income_group', values='debt', aggfunc='mean')

total_income_group,высокий,выше среднего,низкий,очень высокий,очень низкий,средний
children,,,,,,
0,0.1,0.1,0.1,0.1,0.1,0.1
1,0.1,0.1,0.1,0.0,0.1,0.1
2,0.1,0.1,0.1,0.1,0.1,0.1
3,0.1,0.0,0.1,0.0,0.2,0.1
4,0.0,0.3,0.2,nan,nan,0.0
5,nan,0.0,0.0,nan,0.0,0.0


- __Обнаружена зависимость:__ по всем группам с ростом количества детей до 2 включительно растёт доля невозврата кредита в срок. При этом  если детей более 2, то уровень невозврата снижается. Похоже на нормальное распределение, 
- В результатах есть артефакты по уровню невозврата кредитов в срок:
    * в группе с высоким доходом и 4 детьми  — 28,6 %
    * в группе с очень низким доходом и 3 детьми — 22,7 %.
Для уточнения результатов целесообразно провести анализ более широкой выборки.

### Влияние цели кредита на его возврат в срок

In [81]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от ЦЕЛИ КРЕДИТА
print(df.groupby(['purpose_type'])['debt'].mean().sort_values(ascending = False))

purpose_type
покупка автомобиля         0.1
образование                0.1
свадьба                    0.1
операция с недвижимостью   0.1
Name: debt, dtype: float64


In [82]:
# расчёт доли заёмщиков, просрочивших возрат долга, в зависимости от ЦЕЛИ КРЕДИТА
df.pivot_table(index=['income_type'], columns='purpose_type', values='debt', aggfunc='mean')

purpose_type,образование,операция с недвижимостью,покупка автомобиля,свадьба
income_type,,,,
безработный,0.1,0.1,0.1,0.1
госслужащий,0.1,0.0,0.1,0.0
пенсионер,0.1,0.0,0.1,0.1
предприниматель,nan,nan,nan,0.0
сотрудник,0.1,0.1,0.1,0.1
студент,nan,0.0,nan,nan


* Распределение зависимости невозврата кредита в срок от ЦЕЛИ КРЕДИТА довольно плавное. Выбивающихся значений нет.
* Кредиты на операции с недвижимостью стараются выплачивать вовремя — наименьшая просрочка 7,2 % случаев.
* Заёмщики, покупающие автомобиль, наименее финансово дисциплинированны — 9,3 % невозвратов.

## Выводы и рекомендации
<a id = "conclusion"></a>
[В оглавление](#index)

* Семейное положение и количество детей клиента на факт погашения кредита в срок влияет __несистемно__. При выдаче кредита необходимо руководствоваться комплексными метриками. Мне кажется, что эффективнее детализировать все обозначенные метрики с учётом УРОВНЯ ДОХОДА.
* Аномально выская доля невозврата кредитов в срок наблюбдается у групп с высоким доходом и 4 детьми  — 28,6 % и с очень низким доходом и 3 детьми — 22,7 %. Операционному персоналу в отделениях банков необходимо тщательнее проверять финансовое обеспечение потенциальных заёмщиков в группе риска.
* В представленной выборке заёмщиков присутствуют неслучайные аномалии: количество пропущенных строк по СТАЖУ и УРОВНЮ ДОХОДОВ одинаковое и распределены между записями с разным ИСТОЧНИКОМ ДОХОДОВ. __Необходимо отработать эту аномалию пропусков по компетенции.__
* В части записей ПЕНСИОНЕРЫ наблюдается иная аномалия — некорректный стаж. В данном исследовании эти данные непосредственно не используются, но проблему необходимо решить до следующего обращения к выборке. __Необходимо отработать аномалию завышенного стажа у пенсионеров по компетенции.__